# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
vac_df = pd.read_csv("../output_data/Marks_cities.csv")
vac_df.head()

,City,Country,Date,Max Temp,Humidity,Cloudiness,Wind Speed,Lat,Lng
0,Busselton,AU,1587418253,15.00,68,18,4.69,-33.65,115.33
1,Ushuaia,AR,1587417835,6.00,87,75,18.50,-54.80,-68.30
2,Vanimo,PG,1587418534,26.28,82,99,3.01,-2.67,141.30
3,Turaif,SA,1587418535,20.00,37,0,4.10,31.67,38.66
4,Kavieng,PG,1587418535,29.02,71,73,0.24,-2.57,150.80


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)
locations = vac_df[["Lat", "Lng"]].astype(float)
humidity = vac_df["Humidity"].astype(float)
fig = gmaps.figure(center=(0,0), zoom_level=2)
max_humid = vac_df["Humidity"].max()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max_humid,
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
vac_df_F = vac_df.copy()
vac_df_F['Max Temp'] = vac_df_F['Max Temp'] * 9/5 + 32
ideal_df = vac_df_F[vac_df_F['Max Temp'] <= 80]
ideal_df = ideal_df[ideal_df['Max Temp'] >= 65]
ideal_df = ideal_df[ideal_df['Humidity'] <= 55]
ideal_df = ideal_df[ideal_df['Cloudiness'] <= 7]
ideal_df = ideal_df[ideal_df['Wind Speed'] <= 8]

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [44]:
narrowed_city_df = ideal_df.copy()
narrowed_city_df = narrowed_city_df.reset_index()
narrowed_city_df = narrowed_city_df.drop(['index'], axis=1)
narrowed_city_df['Hotel Name'] = ""

target_radius = 5000
target_type = "lodging"
target_search = "Hotel"

for index, row in narrowed_city_df.iterrows():
    hotels_data = []
    target_coordinates = str(row["Lat"]) + "," + str(row["Lng"])
    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }
    
    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    
   # convert response to json
    hotels_data = response.json()

    # Print the json (pretty printed)
    #print(json.dumps(hotels_data, indent=4, sort_keys=True))
    
    try: 
        narrowed_city_df.loc[index, "Hotel Name"] = hotels_data["results"][0]["name"]
    except (IndexError, KeyError, TypeError):
        narrowed_city_df.loc[index, "Hotel Name"] = "N/A"

,City,Country,Date,Max Temp,Humidity,Cloudiness,Wind Speed,Lat,Lng,Hotel Name
0,Turaif,SA,1587418535,68.000,37,0,4.10,31.67,38.66,Golden Dune Hotel
1,Waddān,LY,1587418539,79.790,15,0,4.89,29.16,16.14,N/A
2,Rāwah,IQ,1587418573,75.344,29,2,2.47,34.48,41.91,N/A
3,Baker City,US,1587418586,69.800,22,1,1.50,44.77,-117.83,Geiser Grand Hotel
4,Sabha,LY,1587418590,79.124,19,0,4.55,27.04,14.43,فندق علي
5,Nālūt,LY,1587418594,69.458,31,0,4.72,30.33,10.85,N/A
6,Duncan,US,1587418610,75.002,49,1,2.64,34.50,-97.96,Holiday Inn Express & Suites Duncan
7,São Luiz Gonzaga,BR,1587418618,74.444,42,0,1.88,-28.41,-54.96,Ivos Hotel
8,Santa Maria,BR,1587418651,71.564,53,0,1.04,-29.68,-53.81,Hotel Appel
9,Aswān,EG,1587418654,76.982,23,6,3.71,24.09,32.91,Sofitel Legend Old Cataract Aswan


In [48]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = narrowed_city_df[["Lat", "Lng"]]

In [49]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(marker_layer)

fig
# Display Map

Figure(layout=FigureLayout(height='420px'))